# Project Title

## Table of Contents
1. [Libraries](#1)
2. [Section 2](#2)
3. [Section 3](#3)
4. [Section 4](#4)
5. [Conclusion](#5)

__Introduction__  
This project focuses on ...

__Datasets Used__  
- [Dataset 1](URL)

# 1. Set-Up <a id = '1'></a>

In [1]:
import warnings

import pandas as pd
from scipy.io import arff
from pyspark.sql import SparkSession

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)

In [2]:
# Initialize Spark session with proper configuration
spark = (
    SparkSession.builder.appName("AnomalyDetection")
    .master("local[*]")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "4g")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "true")
    .config(
        "spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true"
    )
    .config(
        "spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true"
    )
    .getOrCreate()
)

# Set log level to reduce verbosity
spark.sparkContext.setLogLevel("WARN")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/02 00:25:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 2. EDA <a id = '2'></a> 

In [3]:
# Read arff file
data, meta = arff.loadarff("../data/bank-additional-ful-nominal.arff")
df = pd.DataFrame(data)
df = df.applymap(lambda x: x.decode("utf-8") if isinstance(x, bytes) else x)

# Examine metadata
print(meta)

# Convert to spark df
bank_df = spark.createDataFrame(df)
bank_df.createOrReplaceTempView("BANK")

bank_df.printSchema()
spark.sql("SELECT * FROM BANK LIMIT 5").show()
bank_df.count()

Dataset: bank-additional-full-weka.filters.unsupervised.attribute.RemoveType-Tnumeric
	job's type is nominal, range is ('housemaid', 'services', 'admin.', 'blue-collar', 'technician', 'retired', 'management', 'unemployed', 'self-employed', 'unknown', 'entrepreneur', 'student')
	marital's type is nominal, range is ('married', 'single', 'divorced', 'unknown')
	education's type is nominal, range is ('basic.4y', 'high.school', 'basic.6y', 'basic.9y', 'professional.course', 'unknown', 'university.degree', 'illiterate')
	default's type is nominal, range is ('no', 'unknown', 'yes')
	housing's type is nominal, range is ('no', 'yes', 'unknown')
	loan's type is nominal, range is ('no', 'yes', 'unknown')
	contact's type is nominal, range is ('telephone', 'cellular')
	month's type is nominal, range is ('may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'mar', 'apr', 'sep')
	day_of_week's type is nominal, range is ('mon', 'tue', 'wed', 'thu', 'fri')
	poutcome's type is nominal, range is ('nonexistent

41188